In [1]:
import os
import json
from pathlib import Path
import pandas as pd
import pymupdf
from tqdm.notebook import tqdm
import random
import jinja2
import string
import textwrap
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
with open("cpv_divisions.json", "r", encoding="utf-8") as file:
    cpv_mapping = json.load(file)

In [3]:
	
current_dir = Path(os.getcwd())
pdf_paths = list(current_dir.rglob("*.pdf"))

data = {
    "file": [],
    "text": [],
    "page": [],
    "division_number": [],
    "division_title": [],
    "language": [],    
}

for p in tqdm(pdf_paths):
    try:
        cft_root = p.parts.index("docs") - 1
        file = "/".join(p.parts[cft_root:])
        
        doc = pymupdf.open(p)

        for idx, page in enumerate(doc):
            data["file"].append(file)
    
            data["text"].append(page.get_text())
        
            data["page"].append(idx+1)
            
            division_number = int(p.parts[p.parts.index("intelliprocure_data") + 1])
            data["division_number"].append(division_number)
        
            division_title = cpv_mapping.get(str(division_number), None)
            data["division_title"].append(division_title)
        
            lang = p.parts[p.parts.index("intelliprocure_data") + 2]
            data["language"].append(lang)

    except:
        print("Could not open file", file)

df = pd.DataFrame(data)
df.head()

  0%|          | 0/1 [00:00<?, ?it/s]

,file,text,page,division_number,division_title,language
0,252518/docs/tender_document_seco_expert_adviso...,\n \nSECO-D-DEAF3401/973\n \nStaatssekretaria...,1,66,Financial and insurance services,EN
1,252518/docs/tender_document_seco_expert_adviso...,\n \n \n2/16\nContent \nAbbreviations...........,2,66,Financial and insurance services,EN
2,252518/docs/tender_document_seco_expert_adviso...,\n \n \n3/16\nAbbreviations \nBöB \n \nBundes...,3,66,Financial and insurance services,EN
3,252518/docs/tender_document_seco_expert_adviso...,\n \n \n4/16\nIntro \n \nThe following instru...,4,66,Financial and insurance services,EN
4,252518/docs/tender_document_seco_expert_adviso...,\n \n \n5/16\nexperiences should sharpen SECO...,5,66,Financial and insurance services,EN


In [8]:
from jinja_helper import process_template


def generate_prompt(text, language):
    option_amt = random.randint(3, 7)
    option_labels = list(string.ascii_uppercase[:option_amt])
    
    context = {
        "option_amt": option_amt,
        "text": text,
        "option_labels": option_labels,
        "language": language
    }
    prompt = process_template('prompts/mcq_prompt.jinja', context)
    return prompt

In [9]:
# takes a list of prompts, sends them to the LLM and returns the chat completions.
def ask_LLM_and_extract_MQC(prompts:list):
    mcq_list = []
    for prompt, sample in prompts:
        load_dotenv()
        deepinfra_api_key = os.getenv("DEEPINFRA_API_KEY")
        deepinfra_base_url = os.getenv("DEEPINFRA_BASE_URL")
        openai = OpenAI(api_key=deepinfra_api_key,
                        base_url=deepinfra_base_url)

        response = openai.chat.completions.create(
                        model="meta-llama/Meta-Llama-3.1-405B-Instruct",
                        messages=[
                            {"role": "system", "content": "You are a knowledgeable assistant that only responds in the given output format."},
                            {"role": "user", "content": prompt}
                        ],
                        max_tokens=1500,
                        temperature=0.7,
                    ).choices[0].message.content.strip()
        mcq_question = extract_MCQ(response)
        if not mcq_question: # give the LLM an oppurtuinity to correct itself
            response = openai.chat.completions.create(
                        model="meta-llama/Meta-Llama-3.1-405B-Instruct",
                        messages=[
                            {"role": "system", "content": "You are a knowledgeable assistant that only responds in the given output format."},
                            {"role": "user", "content": prompt},
                            {"role": "assistant", "content": response},
                            {"role": "user", "content": "I was unable to convert your response into a json file. There is an issue in the format of your response. Please provide the multiple-choice question again, this time in correct JSON Format."}
                        ],
                        max_tokens=1500,
                        temperature=0.7,
                    ).choices[0].message.content.strip()
            mcq_question = extract_MCQ(response)
        if mcq_question:
            mcq_question["language"] = sample['language']
            mcq_question['context'] = sample
            mcq_list.append(mcq_question)
        else:
            print("Unable to provide a MCQ.")

    return mcq_list

def extract_MCQ(response):
    # Extracting MCQ from the answer.
    print(response)
    start = response.find("{")
    end = response.rfind("}")
    if start != -1 and end != -1:  # Ensure both are found
        json_content = response[start:end+1]  
        try:
            mcq_data = json.loads(json_content)
            return mcq_data
        except json.JSONDecodeError:
            return False  
    return False 

def save_to_dict(mcq_list, file_name="LLM_generated_multiple_choice_question.json"):
    path = "output/" + file_name
    with open(path, "w", encoding="utf-8") as f:
        json.dump(mcq_list, f, ensure_ascii=False, indent=4)
    print(f"Multiple-choice question saved to {path}")

In [10]:
def generate_MCQ_for_documents(file_name):
    prompts = []
    samples = df.to_dict(orient='records')
    for sample in samples:
        prompt = generate_prompt(sample['text'], sample['language'])
        prompts.append((prompt, sample))

    mcq_list = ask_LLM_and_extract_MQC(prompts)
    save_to_dict(mcq_list, file_name=file_name)

In [11]:
def shuffle_mcqs(mcq):
    shuffled_mcqs = []
    
    question = mcq['question']
    options = mcq['options']
    correct_answer = mcq['correct_answer']
    language = mcq['language']
    
    for _ in range(4):
        options_values = list(options.values()) 
        random.shuffle(options_values)
        shuffled_options = {label: options_values[i] for i, label in enumerate(options.keys())}
        new_correct_answer = next(label for label, option in shuffled_options.items() if option == options[correct_answer])
    
        shuffled_mcqs.append({
            'question': question,
            'options': shuffled_options,
            'correct_answer': new_correct_answer,
            'version': "reordered",
            'language': language,
            'context': mcq['context']
        })
    
    return shuffled_mcqs

def none_of_the_above_mcqs(mcq):    
    question = mcq['question']
    options = mcq['options']
    correct_answer = mcq['correct_answer']
    language = mcq['language']

    if language == 'DE':
        nota_str = "Keine der oben genannten Optionen"
    else:
        nota_str = "None of the above"
    options_values = list(options.values()) 
    options_values[len(options_values)-1] = nota_str
    options = {label: options_values[i] for i, label in enumerate(options.keys())}
    
    nota_mcq = {
        'question': question,
        'options': options,
        'correct_answer': correct_answer,
        'version': "nota",
        'language': language,
        'context': mcq['context']

    }
    
    return nota_mcq

def true_false(mcq):
    question = mcq['question']
    options = mcq['options']
    correct_answer = mcq['correct_answer']
    language = mcq['language']

    options_values = list(options.values())
    answer = random.choice(options_values)
    label = False
    if options[correct_answer] == answer:
        label = True

    if language == 'DE':
        true_false_prompt = "Ist das folgende Frage-Antwort Paar Richtig oder Falsch?"
    else: 
        true_false_prompt = "Is the following question-answer pair True or False?"

    true_false_question = {
        "true_false_prompt": true_false_prompt,
        "question": question,
        "answer": answer,
        "label": label,
        "version": "true_false",
        "language": language,
        'context': mcq['context']
    }
    return true_false_question

def enhance_mcqs(mcq_list):
    mcqs_plus = []
    for mcq in mcq_list:
        mcq_plus = shuffle_mcqs(mcq)
        mcq_plus[3] = none_of_the_above_mcqs(mcq_plus[3])
        mcq_plus.append(true_false(mcq))
        mcqs_plus.append(mcq_plus)
    return mcqs_plus


In [12]:
file_name = "MCQs.json"

generate_MCQ_for_documents(file_name)

path = "output/" + file_name
with open(path, "r") as f:
    mcq_list = json.load(f)

enhanced_mcqs = enhance_mcqs(mcq_list)
save_to_dict(enhanced_mcqs,file_name="enhanced.json")

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
Here is the multiple-choice question in the given output format:

```
{
    "question": "According to the text, what is the legal basis for the procurement procedure?",
    "options": {
        "A": "Federal Act on Public Procurement of 1 January 2020",
        "B": "Ordinance on Public Procurement of 21 June 2019",
        "C": "Federal Act on Public Procurement of 21 June 2019 and the Ordinance on Public Procurement of 12 February 2020",
        "D": "Federal Act on Public Procurement of 12 February 2020",
        "E": "Ordinance on Public Procurement of 1 January 2020",
        "F": "Federal Act on Public Procurement of 2018"
    },
    "correct_answer": "C"
}
```

Here is the step-by-step reasoning for each option:

* A: Incorrect. The text states that the Feder

KeyboardInterrupt: 

In [13]:
path = "evaluated_german_MCQs_iteration_2.json"
with open(path, "r") as f:
    eval = json.load(f)

correct = eval['correct']
wrong = eval['wrong']
print(len(correct))
print(len(wrong))
all = correct + wrong

answer_counts = {}

for x in correct:
    answer = x['correct_answer']
    if answer in answer_counts:
        answer_counts[answer] += 1
    else:
        answer_counts[answer] = 1

print(answer_counts)

53
8
{'B': 37, 'A': 1, 'C': 13, 'D': 2}
